### mlflow-mvc nedir?
mlflow-mvc, mlflow frameworkünün wrap edilerek `MVC` patterni uygulanmış halidir. Temelde Mlflow'un yapabildiği her fonksiyonaliteye sahiptir. Mlflow'un temel fonksiyonları kullanılarak geliştirilmiştir. Mlflow'un bazı ihtiyaçlara direkt olarak cevap verememesinden dolayı ve dosya yapısının çok karmaşık olmasıdan dolayı geliştirilmiştir. mlflow-mvc standardize bir şekilde geliştirilmiş bir frameworktür. Bu Framework'ün özellikleri:

* Tamamıyla esnektir.
* geliştirmelere sonuna kadar açıktır. Kendi business logic'inizi service katmanında uygulayabilir. Controller katmanında yönetebilirsiniz.
* Dependency injection patterni ile standard hala getirilmiş, bağımlılıklar en aza indirilmiştir.
* Python dinamik bir dil olmasına karşın mlflow-mvc'nin ana komponentleri kısmen opsiyonal olarak type-safety sağlayabilmektedir.

### Başlamadan Önce bilinmesi gerekenler
mlflow-mvc aslında mlflow framework'ünde temel olan 4 Entity baz alınarak geliştirilmiştir. Bu entityler:

* `Run`
* `Experiment`
* `RunVersion`
* `RegisteredRuns`

olarak belirlenmiştir.

Mlflow tamamıyla RDMS olarak 4 Entity üzerine kurulmuştur. Dolayısıyla framework geliştirilirken bu 4 entity baz alınarak `MVC` katmanları oluşturulmuştur. Burada dikkat edilmesi gereken nokta, Run Entity'si aslında `Data` ve `Info` olmak üzere 2 Entity'nin birleşimi olarak ele alınabilir (daha fazla ayrıntı için mlflow dökümantasyonlarına bakabilirsiniz). Single Responsibility prensibi gereği `Run` entitysi 2 ayrı entity'e bölünmüştür.

mlflow-mvc geliştirilirken `spring-boot`'tan etkilenmiştir. `spring-boot` enterprise mimariyi çok iyi yansıtmaktadır. Dolayısıyla dosya yapısı ve mimarisel olarak yüksek düzeyde benzerlikler mevcuttur.

mlflow-mvc geliştirilirken ayrıca yararlanılan 2 temel yardımcı framework vardı

`dependency-injector`: Dependency Injection Design Patternini uygulamak için IoC (Inversion of Controller) Container oluşturmak için tasarlanmış, Cython ile yazılmış mikro bir frameworktür.

`interface`: Python'a Java, c# gibi dillerde mevcut olan interface sınıfını implemente etmeyi sağlar.

### Kütüphanelerin Import Edilmesi
mlflow_mvc adından da anlaşılacağı üzere MVC pattern üzerine inşa edilmiştir. Katmanlar sırasıyla: `Entity Layer` --> `Data Access Layer` (Repository) --> `Service Layer` (Business Logic) --> `Controller Layer`

Bir uygulamada gerekli olabilecek mlflow-mvc paketindeki tüm kütüphaneler aşağıdaki şekilde import edilebilir (pip paketi kurulduktan sonra, ayrıca logger setup .py scriptleri için gerekli değildir, tüm service layer'larına entegre edilmiştir):

In [5]:
import mlflow
import pandas as pd


# Entity Layer
from mlflow_mvc.entities.experiment_entity import ExperimentEntity
from mlflow_mvc.entities.model_version_entity import ModelVersion
from mlflow_mvc.entities._registered_model_entity import RegisteredModelEntity
from mlflow_mvc.entities._run_info_entity import RunInfoEntity
from mlflow_mvc.entities._run_data_entity import RunDataEntity

# Data Access Layer
from mlflow_mvc.repository._experiment_repository import ExperimentRepository
from mlflow_mvc.repository._model_version_repository import ModelVersionRepository
from mlflow_mvc.repository._registered_model_repository import RegisteredModelRepository
from mlflow_mvc.repository._run_repository import RunRepository

# Service Layer
from mlflow_mvc.service._experiment_service import ExperimentService
from mlflow_mvc.service._model_version_service import ModelVersionService
from mlflow_mvc.service._registered_model_service import RegisteredModelService
from mlflow_mvc.service._run_service import RunService

# Controller Layer
from mlflow_mvc.controller._experiment_controller import ExperimentController
from mlflow_mvc.controller._model_version_controller import ModelVersionController
from mlflow_mvc.controller._registered_model_controller import RegisteredModelController
from mlflow_mvc.controller._run_controller import RunController

# Global Configs
from mlflow_mvc.config._core import Config, ApiPath
from mlflow_mvc.util.master_logger import MasterLogger

# Logger Settings
logger = MasterLogger().get_logger
logger.info("execution start..")

[24-02-23, 18:00:25] {1309505168.py - L#35} [INFO] - execution start..


#### Section1: mlflow-mvc Manual Dependency Injection without IoC (Inversion of Control) Container
Bu kısımda logger ayarları ve MLFlow'un experimentation için layerların instanceları yapılır. mlflow-mvc paketi geliştirilirken Dependency injection design patternı uygulandığı için her bir service'in bağımlılıklarını ayrı ayrı instance içinde belirtmek best practice değildir. Yine de Methodoloji olarak uygulanabilir.

#### Repository Instance (Data Access Layer)

In [2]:
TRACKING_SERVER_URI = "http://0.0.0.0:5000"

experiment_repo = ExperimentRepository(tracking_server_url=TRACKING_SERVER_URI)
model_version_repo = ModelVersionRepository(tracking_server_url=TRACKING_SERVER_URI)
registered_models_repo = RegisteredModelRepository(tracking_server_url=TRACKING_SERVER_URI)
run_repo = RunRepository(tracking_server_url=TRACKING_SERVER_URI)

#### Service Instance (Business Logic)

In [3]:
experiment_service = ExperimentService(experiment_repository=experiment_repo)
model_version_service = ModelVersionService(
    run_repository=run_repo,
    model_version_repository=model_version_repo
    )
registered_model_service = RegisteredModelService(registered_models_repository=registered_models_repo)
run_service = RunService(
    experiment_repository=experiment_repo,
    run_repository=run_repo
)

#### Controller Instance

In [4]:
experiment_controller = ExperimentController(experiment_service=experiment_service)
model_version_controller = ModelVersionController(model_version_service=model_version_service)
registered_models_controller = RegisteredModelController(registered_model_service=registered_model_service)
run_controller = RunController(run_service=run_service)

Şimdi Controller katmanımızı da instance ettiğimize göre controller katmanını kullanarak bazı örnekler yapalım:

**Seçilen Metriğe Göre Kaydedilen Son Runin Validasyonu**: Mlflow'a kaydedilen son model için bir validasyon yapalım. Son kaydedilen model dediğimiz için aslında ilgilendiğimiz entity temelde model versiyonudur. Bu yüzden bunun business logic'i `RunVersionController` sınıfı içindedir:

In [5]:
model_version_controller.latest_model_validator(
    model_name="skbc",
    criteria="700",
    selected_metric="epoch")

[23-02-23, 15:12:13] {_model_version_service.py - L#93} [INFO] - _model_version_service.py execution start..
[23-02-23, 15:12:13] {_model_version_service.py - L#99} [INFO] - Registered Model Name: skbc
[23-02-23, 15:12:13] {_model_version_service.py - L#100} [INFO] - Registered Model Description: Edit Model Desc
[23-02-23, 15:12:13] {_model_version_service.py - L#101} [INFO] - Registered Model Run id: 2fc1141d586e41f1ace034398ca99942
[23-02-23, 15:12:13] {_model_version_service.py - L#102} [INFO] - Registered Model Creation Timestamp: 1676551985112
[23-02-23, 15:12:13] {_model_version_service.py - L#103} [INFO] - Registered Model Version: 3
[23-02-23, 15:12:13] {_model_version_service.py - L#104} [INFO] - Registered Model Tags: {'model.tag': 'test', 'model.tag2': 'test2'}
[23-02-23, 15:12:13] {_model_version_service.py - L#105} [INFO] - Registered Model Stage: None
[23-02-23, 15:12:13] {_model_version_service.py - L#113} [INFO] - Metric for latest run --> epoch : 800.0
[23-02-23, 15:12

True

In [6]:
from mlflow_mvc.containers._controller_container import ControllersContainer
container = ControllersContainer()
check = container.model_version_controller().latest_model_validator(
    model_name="skbc",
    criteria="700",
    selected_metric="epoch"
)

[23-02-23, 15:12:16] {_model_version_service.py - L#93} [INFO] - _model_version_service.py execution start..
[23-02-23, 15:12:16] {_model_version_service.py - L#99} [INFO] - Registered Model Name: skbc
[23-02-23, 15:12:16] {_model_version_service.py - L#100} [INFO] - Registered Model Description: Edit Model Desc
[23-02-23, 15:12:16] {_model_version_service.py - L#101} [INFO] - Registered Model Run id: 2fc1141d586e41f1ace034398ca99942
[23-02-23, 15:12:16] {_model_version_service.py - L#102} [INFO] - Registered Model Creation Timestamp: 1676551985112
[23-02-23, 15:12:16] {_model_version_service.py - L#103} [INFO] - Registered Model Version: 3
[23-02-23, 15:12:16] {_model_version_service.py - L#104} [INFO] - Registered Model Tags: {'model.tag': 'test', 'model.tag2': 'test2'}
[23-02-23, 15:12:16] {_model_version_service.py - L#105} [INFO] - Registered Model Stage: None
[23-02-23, 15:12:16] {_model_version_service.py - L#113} [INFO] - Metric for latest run --> epoch : 800.0
[23-02-23, 15:12

**Mlflow'a kaydedilen son modeli indirmek**: Mlflow'a kaydedilen son modeli indirmek için yine aslında `RunVersion` Entitysinden bahsettiğimiz için çağırmamız gereken fonksiyon `RunVersionController` içindedir. Bu fonksiyon 3 parametre alır. Bir tanesi modelin kaydedildiği isim, diğeri folder pathi ve son olarak opsiyonel olarak modelin uzantısını alır. Runin uzantısı default olarak binary format olan `.bin` olarak belirlenmiştir. 

In [6]:
model_version_controller.download_latest_model("skbc", "fasttext_model_path")

[22-02-23, 23:04:56] {_model_version_service.py - L#37} [INFO] - _model_version_service.py execution start..
[22-02-23, 23:04:56] {_model_version_service.py - L#40} [INFO] - latest model is fetching now..
[22-02-23, 23:04:56] {_model_version_service.py - L#44} [INFO] - Model run id : 2fc1141d586e41f1ace034398ca99942
[22-02-23, 23:04:56] {_model_version_service.py - L#45} [INFO] - Model version : 3
[22-02-23, 23:04:56] {_model_version_service.py - L#57} [INFO] - Path to download: result/model/skbc.bin/artifacts/skbc.bin
[22-02-23, 23:04:56] {_model_version_service.py - L#60} [INFO] - Output Directory : /home/ctolon/miniconda3/envs/mlops-latest/lib/python3.8/site-packages/mlflow_mvc/service
[22-02-23, 23:04:56] {_model_version_service.py - L#72} [INFO] - GET Request to -> http://0.0.0.0:5000/get-artifact?path=result/model/skbc.bin/artifacts/skbc.bin&run_uuid=2fc1141d586e41f1ace034398ca99942, params: dict_items([('path', 'result/model/skbc.bin/artifacts/skbc.bin'), ('run_uuid', '2fc1141d5

{'python_function': {'cloudpickle_version': '2.0.0', 'python_model': 'python_model.pkl', 'artifacts': {'fasttext_model_path': {'path': 'artifacts/skbc.bin', 'uri': 'result/model/skbc.bin'}}, 'loader_module': 'mlflow.pyfunc.model', 'python_version': '3.8.15', 'code': 'code', 'env': {'conda': 'conda.yaml', 'virtualenv': 'python_env.yaml'}}}


[22-02-23, 23:05:01] {_model_version_service.py - L#79} [INFO] - skbc download success!


mlflow_mvc paketinde repository altında çeşitli query methodları mevcuttur. MLFlow'da son kaydedilen modelin bilgilerini almak `RunRepository` `find_latest_run_version_by_run_name` adında wraper bir fonksiyon yazılmıştır. Bu fonksiyon parametre olarak modelin ismini alır ve return olarak modelin versiyon bilgisini döner. Bu bilgiden namespace ile `run_uuid` bilgisine ulaşılır ve ileride ele alınacağı üzere run_uuid primary_key olduğundan modeldeki her bir entity'e ulaşabiliriz.

In [7]:
logger.info("latest model is fetching now..")
latest_model = model_version_repo.find_latest_model_version_by_model_name(model_name="skbc")
run_uuid = latest_model.run_id
run_version = latest_model.version
logger.info(f"Run Name: {latest_model.name}")
logger.info(f"Run run id : {run_uuid}")
logger.info(f"Run version : {run_version}")
logger.info(f"Run Creation Timestamp: {latest_model.creation_timestamp}")
logger.info(f"Run Last Updated Timestamp: {latest_model.last_updated_timestamp}")
logger.info(f"Run Current Stage: {latest_model.current_stage}")
logger.info(f"Run Description: {latest_model.description}")
logger.info(f"Run Status: {latest_model.status}")
logger.info(f"Run Tags: {latest_model.tags}")
logger.info(f"Run Status message: {latest_model.status_message}")
logger.info(f"Run Run link: {latest_model.run_link}")
logger.info(f"Run Source: {latest_model.source}")

[23-02-23, 15:12:21] {2887292406.py - L#1} [INFO] - latest model is fetching now..
[23-02-23, 15:12:21] {2887292406.py - L#5} [INFO] - Run Name: skbc
[23-02-23, 15:12:21] {2887292406.py - L#6} [INFO] - Run run id : 2fc1141d586e41f1ace034398ca99942
[23-02-23, 15:12:21] {2887292406.py - L#7} [INFO] - Run version : 3
[23-02-23, 15:12:21] {2887292406.py - L#8} [INFO] - Run Creation Timestamp: 1676551985112
[23-02-23, 15:12:21] {2887292406.py - L#9} [INFO] - Run Last Updated Timestamp: 1676551985112
[23-02-23, 15:12:21] {2887292406.py - L#10} [INFO] - Run Current Stage: None
[23-02-23, 15:12:21] {2887292406.py - L#11} [INFO] - Run Description: Edit Model Desc
[23-02-23, 15:12:21] {2887292406.py - L#12} [INFO] - Run Status: READY
[23-02-23, 15:12:21] {2887292406.py - L#13} [INFO] - Run Tags: {'model.tag': 'test', 'model.tag2': 'test2'}
[23-02-23, 15:12:21] {2887292406.py - L#14} [INFO] - Run Status message: 
[23-02-23, 15:12:21] {2887292406.py - L#15} [INFO] - Run Run link: 
[23-02-23, 15:12

İstersek mlflow-mvc framework'ü yerine mlflow'un kendi API'sini kullanarak örneğin tum `Run` entitylerine ulaşabiliriz:

In [7]:
_ = mlflow.search_runs(
    search_all_experiments=True,
    filter_string='',
    output_format="list",
    )
for i in _:
    print("i = ", i, "type :", type(i))

i =  <Run: data=<RunData: metrics={'dim': 50.0,
 'epoch': 800.0,
 'lr': 0.1,
 'lrUpdateRate': 100.0,
 'wordNgrams': 2.0}, params={}, tags={'PYTHON_VERSION': '3.8.15',
 'mlflow.log-model.history': '[{"run_id": "2fc1141d586e41f1ace034398ca99942", '
                             '"artifact_path": "result/model/skbc.bin", '
                             '"utc_time_created": "2023-02-16 '
                             '12:53:01.633914", "flavors": {"python_function": '
                             '{"cloudpickle_version": "2.0.0", "python_model": '
                             '"python_model.pkl", "artifacts": '
                             '{"fasttext_model_path": {"path": '
                             '"artifacts/skbc.bin", "uri": '
                             '"result/model/skbc.bin"}}, "loader_module": '
                             '"mlflow.pyfunc.model", "python_version": '
                             '"3.8.15", "code": "code", "env": {"conda": '
                             '"conda.y

Benzer şekilde tüm modelleri `pandas` formatında listelemek istiyorsak mlflow içindeki search_runs fonksiyonu kullanılır ve output_format parametresi `pandas` olarak değiştirilir.

In [8]:
pd.pandas.set_option('display.max_columns', None)
pandas_format = mlflow.search_runs(
    search_all_experiments=True,
    output_format="pandas"
    )
pandas_format.head()

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.epoch,metrics.lrUpdateRate,metrics.lr,metrics.dim,metrics.wordNgrams,metrics.mae,params.max_depth,tags.mlflow.note.content,tags.mlflow.user,tags.mlflow.source.type,tags.release.candidate,tags.mlflow.project.env,tags.runtag,tags.release.version,tags.mlflow.source.git.commit,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.mlflow.runName,tags.PYTHON_VERSION
0,2fc1141d586e41f1ace034398ca99942,1,FINISHED,http://localhost:8001/api/2.0/mlflow-artifacts...,2023-02-16 12:53:00.351000+00:00,2023-02-16 12:53:05.121000+00:00,800.0,100.0,0.1,50.0,2.0,NaN,None,Edit Run Desc,ctolon,LOCAL,RC1,docker,runtag,2.2.0,6ef5e818217673f7fa45ed51f2ab7b61589cf813,"[{""run_id"": ""2fc1141d586e41f1ace034398ca99942""...",train.py,deneme1,3.8.15
1,6fd4e1083531476586560ef752f97dac,1,FINISHED,http://localhost:8001/api/2.0/mlflow-artifacts...,2023-02-16 11:23:04.322000+00:00,2023-02-16 11:23:09.384000+00:00,800.0,100.0,0.1,50.0,2.0,NaN,None,Edit Run Desc,ctolon,LOCAL,RC1,docker,runtag,2.2.0,6ef5e818217673f7fa45ed51f2ab7b61589cf813,"[{""run_id"": ""6fd4e1083531476586560ef752f97dac""...",train.py,deneme1,3.8.15
2,02c7eff999714ef0b4cfe1026423d443,4,FINISHED,http://localhost:8001/api/2.0/mlflow-artifacts...,1970-01-20 09:43:53.913000+00:00,1970-01-20 09:43:54.913000+00:00,NaN,NaN,NaN,NaN,NaN,0.99,2,None,None,None,None,None,None,None,None,None,None,run_for_exp_py_exp_16766339136901524,None
3,d98f9ddb31884908b02fdbe4295990f4,3,FINISHED,http://localhost:8001/api/2.0/mlflow-artifacts...,1970-01-20 09:43:51.270000+00:00,1970-01-20 09:43:52.270000+00:00,NaN,NaN,NaN,NaN,NaN,0.99,2,None,None,None,None,None,None,None,None,None,None,run_for_exp_py_exp_16766312703932707,None
4,7bb3a850c4d943938d6371f9de510442,2,FINISHED,http://localhost:8001/api/2.0/mlflow-artifacts...,1970-01-20 09:43:51.173000+00:00,1970-01-20 09:43:52.173000+00:00,NaN,NaN,NaN,NaN,NaN,0.99,2,None,None,None,None,None,None,None,None,None,None,run_for_exp_py_exp_16766311733228045,None


In [9]:
pandas_format.columns.to_list()

['run_id',
 'experiment_id',
 'status',
 'artifact_uri',
 'start_time',
 'end_time',
 'metrics.epoch',
 'metrics.lrUpdateRate',
 'metrics.lr',
 'metrics.dim',
 'metrics.wordNgrams',
 'metrics.mae',
 'params.max_depth',
 'tags.mlflow.note.content',
 'tags.mlflow.user',
 'tags.mlflow.source.type',
 'tags.release.candidate',
 'tags.mlflow.project.env',
 'tags.runtag',
 'tags.release.version',
 'tags.mlflow.source.git.commit',
 'tags.mlflow.log-model.history',
 'tags.mlflow.source.name',
 'tags.mlflow.runName',
 'tags.PYTHON_VERSION']

#### Section 6: Runin Metadalarına JSON format çıktısında erişmek
Run Entity'sinin metadatalarına daha kolay erişmek için `RunDataEntity` Adında bir class yazılmıştır. Bu class recursive olarak entity'nin her bir field'ına `get` metoduyla ulaşılarak modeldeki her bir niceliğe key - value yaklaşımıyla ulaşır.

In [10]:
# Get Run Run by Run Id
run = run_repo.find_run_by_run_id(run_id=run_uuid)
run_data_model = RunDataEntity(run, "fasttext_model_path", strict_return=True)
run_info_model = RunInfoEntity(run)

Örneğin modelimizin response'unu inceleyelim:

In [11]:
run, type(run)

(<Run: data=<RunData: metrics={'dim': 50.0,
  'epoch': 800.0,
  'lr': 0.1,
  'lrUpdateRate': 100.0,
  'wordNgrams': 2.0}, params={}, tags={'PYTHON_VERSION': '3.8.15',
  'mlflow.log-model.history': '[{"run_id": "2fc1141d586e41f1ace034398ca99942", '
                              '"artifact_path": "result/model/skbc.bin", '
                              '"utc_time_created": "2023-02-16 '
                              '12:53:01.633914", "flavors": {"python_function": '
                              '{"cloudpickle_version": "2.0.0", "python_model": '
                              '"python_model.pkl", "artifacts": '
                              '{"fasttext_model_path": {"path": '
                              '"artifacts/skbc.bin", "uri": '
                              '"result/model/skbc.bin"}}, "loader_module": '
                              '"mlflow.pyfunc.model", "python_version": '
                              '"3.8.15", "code": "code", "env": {"conda": '
                           

Görüldüğü üzere modelin response'u mlflow'da generic bir tip olan `Run` olarak dönüyor. Bunun için bir `dict` transformation yazılmıştır ve entity methodlarında `__init__` yani constructor tarafında uygulanmıştır. Bu transformasyon response'un çıktısını preprocessing ile işlenebilir hale getirir. Bu transformation bu şekildedir:

In [12]:
dict(run)

{'data': <RunData: metrics={'dim': 50.0,
  'epoch': 800.0,
  'lr': 0.1,
  'lrUpdateRate': 100.0,
  'wordNgrams': 2.0}, params={}, tags={'PYTHON_VERSION': '3.8.15',
  'mlflow.log-model.history': '[{"run_id": "2fc1141d586e41f1ace034398ca99942", '
                              '"artifact_path": "result/model/skbc.bin", '
                              '"utc_time_created": "2023-02-16 '
                              '12:53:01.633914", "flavors": {"python_function": '
                              '{"cloudpickle_version": "2.0.0", "python_model": '
                              '"python_model.pkl", "artifacts": '
                              '{"fasttext_model_path": {"path": '
                              '"artifacts/skbc.bin", "uri": '
                              '"result/model/skbc.bin"}}, "loader_module": '
                              '"mlflow.pyfunc.model", "python_version": '
                              '"3.8.15", "code": "code", "env": {"conda": '
                              

Görüldüğü üzere artık sonuç daha anlamlıdır. Transformation uygulanmış response'u incelersek dataların key - value şeklinde tutulduğu, `mlflow.log-model.history` fieldı'nın valuesi ise `json` tipinde dönmüştür! Bunu ayrı olarak ele almak gerekir. Bu durum `RunDataEntity` sınıfı içinde özel olarak ele alınmıştır.

In [13]:
# Debug
c = run_data_model.get_model_log_history
c["flavors"]
c["flavors"]["python_function"]["code"]

'code'

Şimdi `Run` Entitysinin mevcut tüm fieldlarına `get` metodları ile ulaşabiliriz

In [15]:
# Properties
run_metrics = run_data_model.get_metrics
run_params = run_data_model.get_params
run_tags = run_data_model.get_run_tags
run_source_name = run_data_model.get_source_name
run_source_type = run_data_model.get_source_type
run_git_commit = run_data_model.get_git_commit
run_desc = run_data_model.get_description
run_run_name = run_data_model.get_run_name
run_project_env = run_data_model.get_project_env
run_meta_data = run_data_model.get_model_log_history
run_run_id = run_data_model.get_run_id
run_artifact_path = run_data_model.get_artifact_path
run_creation_time = run_data_model.get_utc_time_created
run_flavors = run_data_model.get_flavors
run_py_func = run_data_model.get_python_function
run_artifacts = run_data_model.get_artifacts
run_path_and_uri = run_data_model.get_model_path_and_uri
run_path = run_data_model.get_model_path 
run_uri = run_data_model.get_model_uri
run_loader_module = run_data_model.get_loader_module
run_python_version = run_data_model.get_python_version
run_code = run_data_model.get_code
run_env = run_data_model.get_model_env
run_conda_env = run_data_model.get_conda_env
run_virtual_env = run_data_model.get_virtual_env
run_uuid = run_data_model.get_model_uuid
run_mlflow_version = run_data_model.get_mlflow_version

# Displays
print("Run Metrics: ", run_metrics, type(run_metrics))
print("Run Params :", run_params, type(run_params))
print("Run Tags: ", run_tags, type(run_tags))
print("Run Source name: ", run_source_name, type(run_source_name))
print("Run Source Type: ", run_source_type, type(run_source_type))
print("Run Git Commit: ", run_git_commit, type(run_git_commit))
print("Run Description: ", run_desc, type(run_desc))
print("Run Run name: ", run_run_name, type(run_run_name))
print("Run Environment ", run_project_env, type(run_project_env))
print("Run Meta Data :", run_meta_data, type(run_meta_data))
print("Run Run Id :", run_run_id, type(run_run_id))
print("Run Artifact Path :", run_artifact_path, type(run_artifact_path))
print("Run Creation Time in UTC :", run_creation_time, run_creation_time, type(run_creation_time))
print("Run Flavors: ", run_flavors, type(run_flavors))
print("Run Python Function :", run_py_func, type(run_py_func))
print("Run Artifacts: ", run_artifacts, type(run_artifacts))
print("Run Path and URI:", run_path_and_uri, type(run_path_and_uri))
print("Run Path: ", run_path, type(run_path))
print("Run URI :", run_uri, type(run_uri))
print("Run Loader Module :", run_loader_module, type(run_loader_module))
print("Run Python Version :", type(run_python_version))
print("Run Code :", run_code, type(run_code))
print("Run Env :", run_env, type(run_env))
print("Run Conda Env :", run_conda_env, type(run_conda_env))
print("Run Virtual Env :", run_virtual_env, type(run_virtual_env))
print("Run UUID :", type(run_uuid))
print("Run MLflow Version :", type(run_mlflow_version))

Run Metrics:  {'epoch': 800.0, 'dim': 50.0, 'wordNgrams': 2.0, 'lr': 0.1, 'lrUpdateRate': 100.0} <class 'dict'>
Run Params : {} <class 'dict'>
Run Tags:  {'mlflow.user': 'ctolon', 'mlflow.source.name': 'train.py', 'mlflow.source.type': 'LOCAL', 'mlflow.source.git.commit': '6ef5e818217673f7fa45ed51f2ab7b61589cf813', 'runtag': 'runtag', 'release.candidate': 'RC1', 'release.version': '2.2.0', 'mlflow.note.content': 'Edit Run Desc', 'mlflow.runName': 'deneme1', 'mlflow.project.env': 'docker', 'PYTHON_VERSION': '3.8.15', 'mlflow.log-model.history': '[{"run_id": "2fc1141d586e41f1ace034398ca99942", "artifact_path": "result/model/skbc.bin", "utc_time_created": "2023-02-16 12:53:01.633914", "flavors": {"python_function": {"cloudpickle_version": "2.0.0", "python_model": "python_model.pkl", "artifacts": {"fasttext_model_path": {"path": "artifacts/skbc.bin", "uri": "result/model/skbc.bin"}}, "loader_module": "mlflow.pyfunc.model", "python_version": "3.8.15", "code": "code", "env": {"conda": "conda

In [16]:
run_info_uuid = run_info_model.get_run_uuid
run_info_uuid

'2fc1141d586e41f1ace034398ca99942'